In [ ]:
from pyspark.sql.functions import *
import string
import string
# nltk.download()
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords



In [14]:
full_data = spark.read.option("header", "true"). \
    option("mode", "DROPMALFORMED") \
    .csv(r"C:\Users\sgoel\PycharmProjects\University\BigData\Targil_3\Reviews_1400.csv")



In [15]:
full_data = full_data.select('Score', 'Summary').dropna()


In [16]:

# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet
from pyspark.sql.functions import udf

def splitForWordnet(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    


lemma = WordNetLemmatizer()
analyzer = SentimentIntensityAnalyzer() 


def cleanText(text_data):
    tokens = word_tokenize(text_data)
    tokens = [word.lower() for word in tokens]

    punc = str.maketrans('', '', string.punctuation)
    replaced = [word.translate(punc) for word in tokens]
    all_stop_words = set(stopwords.words('english'))
    text_data = [word for word in replaced if word not in all_stop_words]
    

    text_data = [t for t in text_data if len(t) > 0]
    pos_tags = pos_tag(text_data)
    text_data = [lemma.lemmatize(t[0], splitForWordnet(t[1])) for t in pos_tags]
    text_data = [t for t in text_data if len(t) > 1]


    return text_data

def vader(data_to_analyze):
    data_to_analyze = " ".join(data_to_analyze)
    score =  analyzer.polarity_scores(data_to_analyze)
    return score['compound']

clean_udf = udf(cleanText())
vader_udf = udf(vader)

data = full_data.withColumn("text_review", clean_udf(col("Summary")))
data = data.withColumn("score_review", vader_udf(col("text_review")))
data = data.withColumn("label", when(col("description")>80, 1.0).otherwise(0.0))
data = data.select(["Summary","label","score_review"])

positiveScore = data.filter((col("score_review") >= 0.75))
negetiveScore = data.filter((col("score_review") <= -0.75))        
irony = data.filter((col("score_review") <= -0.75) & (col("label") == 1))    




NameError: name 'SentimentIntensityAnalyzer' is not defined

In [18]:

irony = irony.rdd
irony.collect()

negetiveScore = negetiveScore.rdd
negetiveScore.collect()

positiveScore = positiveScore.rdd
positiveScore.collect()

NameError: name 'irony' is not defined